##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Guide to using Gemma 2 with Langfun, PyGlove and llama.cpp

[Langfun](https://github.com/google/langfun) is a versatile library designed to bridge the gap between natural language processing (NLP) and structured data manipulation. It allows you to effortlessly parse, interpret, and convert unstructured text into well-defined Python objects. This capability is invaluable for applications that require seamless integration between language models and data-driven systems.

[PyGlove](https://github.com/google/pyglove) is a powerful library for flexible and scalable configuration management and automated machine learning (AutoML). It provides tools for defining complex schemas and object representations, enabling you to create highly customizable and maintainable code structures. PyGlove's compatibility with Langfun ensures that the objects you generate from natural language prompts are both robust and easy to use.

[llama.cpp](https://github.com/ggerganov/llama.cpp) is a C++ implementation of Meta AI's LLaMA and other large language model architectures, designed for efficient performance on local machines or within environments like Google Colab. It enables you to run large language models without needing extensive computational resources.

[Gemma](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open-source language models from Google. Built from the same research and technology used to create the Gemini models, Gemma models are text-to-text, decoder-only large language models (LLMs), available in English, with open weights, pre-trained variants, and instruction-tuned variants.
Gemma models are well-suited for various text-generation tasks, including question-answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop, or your cloud infrastructure, democratizing access to state-of-the-art AI models and helping foster innovation for everyone.

By combining `Langfun`, `PyGlove`, `llama.cpp` and `Gemma`, you can create applications that not only understand and process natural language but also convert that understanding into structured, actionable data. This integration allows you to build sophisticated systems that can interact with external APIs, databases, and other services using well-defined objects, streamlining workflows and enhancing functionality.

In this guide, you'll learn how to set up your environment, configure the Gemma 2 model, and implement various real-world use cases.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Gemma_with_Langfun_and_LlamaCpp.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

**Before you dive into the tutorial, let's get you set up with Gemma:**

1. **Hugging Face Account:**  If you don't already have one, you can create a free Hugging Face account by clicking [here](https://huggingface.co/join).
2. **Gemma Model Access:** Head over to the [Gemma model page](https://huggingface.co/collections/google/gemma-2-release-667d6600fd5220e7b967f315) and accept the usage conditions.
3. **Colab with Gemma Power:**  For this tutorial, you'll need a Colab runtime with enough resources to handle the Gemma 9B model. Choose an appropriate runtime when starting your Colab session.
4. **Hugging Face Token:**  Generate a Hugging Face access (preferably `write` permission) token by clicking [here](https://huggingface.co/settings/tokens). You'll need this token later in the tutorial.

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**


### Configure your HF token

Add your Hugging Face token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

You'll need to install a few Python packages for Langfun and PyGlove and some dependencies to interact with HuggingFace along with `llama.cpp` and run the model.

Run the following cell to install or upgrade it:

In [ ]:
# Install ccache, a compiler cache to speed up compilations
!apt install -y ccache

# Install Langfun and PyGlove Python libraries quietly
!pip install -q langfun pyglove

# Upgrade and install the huggingface_hub library for interacting with Hugging Face
!pip install --upgrade -q huggingface_hub

# Clone the llama.cpp repository from GitHub
!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp.git

# Navigate into the llama.cpp directory and compile the project with CUDA support
!cd llama.cpp && make -j8 GGML_CUDA=1

You are compiling the `llama.cpp` binary, which may take some time. Please keep your browser tab active to prevent Colab from shutting down due to inactivity, and return once the compilation is complete to resume working with the notebook.

### Logging into Hugging Face Hub

Next, to authenticate and gain access to download the Gemma 2 model, you need to log in to the Hugging Face Hub using your access token.

In [ ]:
from huggingface_hub import login

login(os.environ["HF_TOKEN"])

### Downloading the Gemma 2 Model
Once you're logged in, you can download the Gemma 2 model files from Hugging Face. The [Gemma 2 model](https://huggingface.co/bartowski/gemma-2-9b-it-GGUF) is available in **GGUF** format, which is optimized for use with `llama.cpp`.


In [ ]:
from huggingface_hub import hf_hub_download

# Specify the repository and filename
repo_id = 'bartowski/gemma-2-9b-it-GGUF'  # Repository containing the GGUF model
filename = 'gemma-2-9b-it-Q6_K.gguf'        # The GGUF model file

# Download the model file to the current directory
hf_hub_download(repo_id=repo_id, filename=filename, local_dir='.')

### Running the Gemma 2 model using `llama.cpp`

You will initialize the model using the `llama.cpp` library by first loading a pre-trained Gemma 2 model from HuggingFace. Next, you'll prompt the model by asking a simple question so that you can test if `llama.cpp` is working with the model that you just downloaded.

In [ ]:
!# Define your prompt
prompt = "Q: I have 3 apples and I eat 2. How many apples do I have left?\nA:"
model_path = "gemma-2-9b-it-Q6_K.gguf"

# Run the model using llama.cpp
!./llama.cpp/llama-cli -m "{model_path}" -p "{prompt}" \
  --color --chat-template gemma --temp 0.7 -ngl 9999

Next, you'll run the server with `llama-server` to serve the Gemma 2 model, allowing for API-based interactions.

In [ ]:
!./llama.cpp/llama-server \
  --model "{model_path}" \
  --chat-template gemma \
  --host 0.0.0.0 \
  --port 8000 \
  --chat-template gemma \
  --n-gpu-layers 9999 > server.log 2>&1 &

# Add a delay to allow the server to warm up before making any API requests to
# the server
!sleep 30

Hint: You can use `tail -f server.log` to preview the command's output if necessary.

In [ ]:
from google.colab.output import eval_js

# (Optional) View the llama.cpp server interface
# Uncomment the line below to generate a URL for the server
# print(eval_js("google.colab.kernel.proxyPort(8000)"))

Note: Uncommenting the print statement will display the proxied URL, which you can open in a new browser tab to interact with the server.

### Import Libraries

Import the necessary Python libraries for `Langfun` and `PyGlove`, which are essential for defining schemas and interacting with the language model. These libraries provide the foundational tools needed to define data schemas and query the language model effectively.

In [ ]:
import langfun as lf
import pyglove as pg

To programmatically add control tokens to any prompt for use with an instruction-tuned model, you can create a function that wraps the prompt with the necessary control tokens. Here’s an example Python function that can be reused to apply this formatting:

In [ ]:
def format_gemma_prompt(user_input):
    """
    Formats a prompt for the Gemma instruction-tuned model by adding control tokens.
    Here you'll be instructing the model to adhere to Langfun and PyGlove.

    Parameters:
    user_input (str): The input from the user to be formatted.

    Returns:
    str: The formatted prompt with Gemma control tokens.
    """
    return f"""\
    You are an AI assistant designed to work seamlessly with Langfun and PyGlove. Your task is to parse user inputs into complex object representations compatible with these libraries. Always output as an object and nothing else. Do not include any additional text, explanations, or comments.

    Guidelines:
    Output Format: Always provide your output as an object representation suitable for Langfun and PyGlove.
    Consistency: Do not deviate from producing the object. Ensure your output is valid and can be used directly in code.
    No Extra Text: Exclude any additional explanations or comments outside of the object.

    Technical Details:
    Your goal is to help parse user inputs into object representations that can be directly utilized within Langfun and PyGlove code.

    <start_of_turn>user
    {user_input}
    <end_of_turn>
    <start_of_turn>model
    """

### Using Langfun with Gemma 2 from Google AI

Langfun allows you to define a clear schema and effortlessly convert natural language inputs into structured objects. These objects can then be directly plugged into external systems, databases, or external APIs, facilitating seamless data flow and integration without the need for manual parsing or data cleaning.

First, let's try a simple use case by asking Langfun to represent a simple mathematical statement's final answer using a Python class `Answer`.


In [ ]:
class Answer(pg.Object):
  result: int

prompt = 'The result of one plus two is three'
result = lf.query(
    prompt=format_gemma_prompt(prompt),
    schema=Answer,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

To break down what's happening here.

* `lf.query` first sends the formatted prompt to the language model and parses the response into the specified schema.

* `format_gemma_prompt(prompt)` formats the prompt by adding necessary control tokens and instructions.

* `schema=Answer` specifies that the output should conform to the `Answer` schema.

* `lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")` makes Langfun use the local `llama.cpp` server running the Gemma 2 9B IT quantized model.

Next, you'll take a look at real-world use cases demonstrating how you can leverage Langfun and Gemma 2 to build robust applications.

### Parsing a Job Description into a Structured Object


Automating the recruitment process can significantly enhance efficiency and accuracy in HR departments. By parsing job descriptions into structured objects, you can seamlessly integrate this data into your HR systems, enabling functionalities like resume matching, job postings, and analytics.

Let's start first by defining a `Job` schema using PyGlove, then provide a job description as input. Langfun, powered by the Gemma 2 model, processes the text and extracts relevant information, populating the `Job` object accordingly.


In [ ]:
# Define the Job schema
class Job(pg.Object):
    title: str
    company: str
    location: str
    responsibilities: list[str]
    requirements: list[str]

# Job description text
job_description = """
We are looking for a Software Engineer to join our team at TechCorp in San Francisco.
Responsibilities include developing software solutions, collaborating with cross-functional teams, and participating in code reviews.
Requirements: Bachelor's degree in Computer Science, 3+ years of experience in software development, proficiency in Python and JavaScript.
"""

# Parse the job description
result = lf.query(
    prompt=format_gemma_prompt(job_description),
    schema=Job,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

This structured data can then be used directly in your HR systems or external APIs for various purposes such as job matching and integration with Applicant Tracking Systems (ATS) for streamlined recruitment workflows.

### Enabling Automated Handling of Support Tickets

In this second example, you'll create a support ticket by incorporating variables into your prompt. In customer service applications, generating support tickets efficiently is crucial for timely issue resolution. By creating structured support tickets from user inputs, you can integrate seamlessly with ticketing systems, prioritize issues, and assign them to the appropriate teams.

Define a `SupportTicket` schema and use variables like `customer_name` and `issue` in your prompt. Langfun processes these variables within the context, generating a structured `SupportTicket` object that can be integrated into your customer service systems or external APIs. This lets you automatically create tickets in a system, easily track ticket statuses and generate reports.

In [ ]:
customer_name = 'John Doe'
issue = 'I am unable to log into my account.'

In [ ]:
# Define the SupportTicket schema
class SupportTicket(pg.Object):
    customer_name: str
    issue: str
    steps_taken: list[str]
    resolution: str | None

# Prompt with variables
prompt = 'Create a support ticket for {{customer_name}} who reported: "{{issue}}" and propose steps to be taken along with a final resolution'

# Generate the support ticket
result = lf.query(
    prompt=format_gemma_prompt(prompt),
    customer_name=customer_name,
    issue=issue,
    schema=SupportTicket,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

In a scenario where your customer service team receives numerous inquiries daily, by automating the creation of support tickets, you can ensure that each issue is documented consistently and efficiently. The structured `SupportTicket` objects can be directly fed into your CRM or ticketing software, facilitating faster response times and better customer satisfaction.


### Parsing Customer Feedback into Structured Data

Understanding customer feedback is essential for improving products and services. You'll start by first defining a `Feedback` schema and provide customer feedback text. Langfun processes the feedback, determining sentiment and identifying key topics, and outputs a structured `Feedback` object. This allows you to parse customer feedback into structured data, aiding in sentiment analysis, report generation and topic modeling.

In [ ]:
# Define the Feedback schema
from typing import Literal

class Feedback(pg.Object):
    customer_id: int
    feedback_text: str
    sentiment: Literal['positive', 'negative', 'mixed']
    topics: list[str]

# Customer feedback text
feedback_text = 'I love the new features in the app, but it crashes frequently when I try to upload photos.'

# Context for the model
context = f'Parse the following customer (id: 1) feedback into an object representation (Feedback): {feedback_text}'

# Parse the feedback
result = lf.query(
    prompt=format_gemma_prompt(context),
    schema=Feedback,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

This example shows how the model can extract valuable insights from customer feedback, which is crucial for product improvement. Imagine you manage a mobile app with a large user base. By systematically parsing feedback, you can identify which features are well-received and which areas require attention. For instance, detecting frequent crashes related to photo uploads allows your development team to prioritize bug fixes, enhancing the overall user experience.


### Classifying Support Tickets Using `Literal` Types

Efficiently categorizing and prioritizing support tickets can significantly enhance your ticket triaging system. Proper classification ensures that issues are addressed by the right teams promptly, improving resolution times and customer satisfaction.

Define a `Ticket` schema with specific categories and priorities. Provide a support ticket description, and Langfun processes it to classify the ticket accordingly. Next, let's perform classification tasks using `Literal` types from the `typing` module, which is useful for ticket triaging systems.

In [ ]:
# Define the Ticket schema
class Ticket(pg.Object):
    id: int
    issue: str
    priority: Literal['low', 'medium', 'high']
    category: Literal['billing', 'technical', 'account', 'other']
    assigned_to: str | None

# Support ticket description
ticket_description = 'Ticket ID: 12345. Customer reports that they are unable to update their billing information on the website.'

# Context for classification
context = f'Classify the following support ticket into the Ticket object: {ticket_description}'

# Classify the ticket
result = lf.query(
    prompt=format_gemma_prompt(context),
    schema=Ticket,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

In a customer service center, tickets often vary widely in nature and urgency. By automatically classifying tickets, you ensure that billing issues are handled by a finance team, technical problems by the IT department, and so forth. Prioritizing tickets based on their severity helps in managing workloads effectively and ensuring that high-priority issues are resolved swiftly. This demonstrates the model's ability to categorize and prioritize support tickets automatically.


###  Chain-of-Thought Reasoning

Chain-of-thought (CoT) reasoning involves breaking down complex problems into a series of logical steps. This approach enhances the model's ability to solve intricate tasks by systematically reasoning through each component.

In this example, you'll demonstrate the model's capability in performing CoT reasoning for a simple puzzle.


In [ ]:
from typing import Optional, List

class Step(pg.Object):
  description: str
  step_output: float

class Solution(pg.Object):
  question: str
  steps: list[Step]
  final_answer: int


# Puzzle question
question = (
  'Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. '
  'She sells the remainder at the farmers\' market daily for $2 per fresh duck egg. '
  'How much in dollars does she make every day at the farmers\' market? '
)

# Assume gemma2_llm is your initialized Gemma 9b model
result = lf.query(
    prompt=format_gemma_prompt(question),
    schema=Solution,
    lm=lf.llms.LlamaCppRemote("http://0.0.0.0:8000")
)
print(result)

This demonstrates the model's capability in performing chain-of-thought reasoning.


In this notebook, you've explored how to leverage **Langfun**, **PyGlove**, **llama.cpp** and the **Gemma 2** model from **Hugging Face** to address a variety of real-world scenarios. Through practical use cases—including parsing job descriptions, creating support tickets, analyzing customer feedback, classifying support tickets, and performing chain-of-thought reasoning—you've seen how Langfun effortlessly converts unstructured text into structured, actionable objects. This showcases the model's effectiveness in real-world scenarios, enhancing the capabilities of your applications and seamlessly integrating with external systems.

Feel free to extend these examples to fit your specific needs and explore the full potential of Langfun and Gemma in your projects!
